# Baselines + GBM


Baselines: persistence (t‑24) and moving average vs. GBM baseline.

In [ ]:
# Environment setup
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)

In [ ]:
import subprocess

subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.build_features',
    '--in', 'data/raw', '--out', 'data/processed'
], check=True)
subprocess.run([
    'python', '-m', 'gridpulse.data_pipeline.split_time_series',
    '--in', 'data/processed/features.parquet',
    '--out', 'data/processed/splits'
], check=True)


In [ ]:
import subprocess

for target in ['load_mw', 'wind_mw', 'solar_mw']:
    subprocess.run([
        'python', '-m', 'gridpulse.forecasting.train_baseline',
        '--features', 'data/processed/features.parquet',
        '--splits', 'data/processed/splits',
        '--target', target
    ], check=True)


In [ ]:
import json
from pathlib import Path

for p in Path('artifacts/backtests').glob('week1_baseline_*.json'):
    print(p.name)
    print(json.loads(p.read_text()))
